In [58]:
k = GF(3)
R.<T> = PolynomialRing(k)

F = T^11 -1

I = R.ideal([F])

A= R.quotient(I)

The smallest $j$ for which that $\mathbb{F}_{3^j}$ contains a root of $f$ is $j=5$:

In [14]:
[ (i,mod(3^i -1,11)) for i in range(6) ]

[(0, 0), (1, 2), (2, 8), (3, 4), (4, 3), (5, 0)]

Now, the field $\ell=\mathbb{F}_{3^5}$ has multiplicative group of order $3^5 - 1 = 242$:

In [17]:
3^5 - 1

242

Let's get a generator for the multiplicative group $\ell^\times = \mathbb{F}_{3^5}^\times$:

In [24]:
l.<z> = GF(3^5)
multiplicative_order(z)

242

Using `z` we can find a primitive 11th root of unity in $\ell$:

In [26]:
a = z^(242/11)
multiplicative_order(a)

11

Now, the minimal polynomial of `a` over $\mathbb{F}_3$ must have degree 5. What are its roots?

Well, if `a` is a root, also `a^3` is a root. Similary, `a^(3^i)` is a root for every natural number `i`.

In [57]:
[( f"a^(3^{i})" , a^(3^i) ) for i in range(6)]

[('a^(3^0)', 2*z^3 + 2*z^2 + z + 1),
 ('a^(3^1)', z^4 + z^3 + 2*z^2 + 2),
 ('a^(3^2)', z^3 + 2*z + 1),
 ('a^(3^3)', 2*z^4 + 2*z^3 + z),
 ('a^(3^4)', 2*z^2 + 2*z + 1),
 ('a^(3^5)', 2*z^3 + 2*z^2 + z + 1)]

In [62]:
# the monic minimal polynomial is the product of the linear factors corresponding to these roots. Namely

g = product([T - a^(3^i) for i in range(5)])
g

T^5 + T^4 + 2*T^3 + T^2 + 2

Note that this polynomial has coefficients in $\mathbb{F}_3$!

In addition to the trivial root $1$, there are still another 5 roots to $T^{11} - 1$ in $\ell = \mathbb{F}_{3^5}$ to account for.

Well, the roots of $T^{11} - 1$ in $\ell$ are precisely the elements of the multiplicative subgroup $\langle a \rangle$.

So what we really need is a good description of the roots of $f$.

So we need to describe the set of exponents $1,3,3^2,3^3,3^4$ modulo $11$.

In [35]:
[ mod(3^i,11) for i in range(5) ]

[1, 3, 9, 5, 4]

This shows that the roots of $f$ are exactly $a^i$ for $i$ in `S=[1, 3, 9, 5, 4]`

The set `S` is called a *cyclotomic coset*.

Note that $a^2$ is a root of $T^11 - 1$ but is not a root of $f$. So we need to consider the cyclotomic coset containing 2, namely:

In [37]:
[ mod(2*3^i,11) for i in range(5) ]

[2, 6, 7, 10, 8]

The minimal polynomial $g$ of $a^2$ has roots $a^j$ for $j$ in [2, 6, 7, 10, 8].

In [61]:
S1=[ mod(2*3^i,11) for i in range(5) ]
h = product([T - a^j for j in S1])
h

T^5 + 2*T^3 + T^2 + 2*T + 2

Since `g` and `fh have no roots in common, their `gcd` is 1.

Let's check that we have factored $T^{11} - 1$

In [63]:
(T-1)*g*h == T^11 -1

True

In [44]:
parent(f)

Univariate Polynomial Ring in T over Finite Field in z of size 3^5

In [64]:
## as constructed, sage knows g and h to have coefficients in F_3^5, even though they
## are really in F_3. We ask for versions whose coefficients are known to be in F_3:
gg = R(g)
hh = R(h)
[ gg.is_irreducible(), hh.is_irreducible() ]

[True, True]

In [50]:
(T-1)*ff*gg == T^11 - 1

True

Now, let us notice that `g` and `h` are related by:
$h(T) = -T^5 g(1/T)$.

In [65]:
h == -T^5*g(T = 1/T)

True

Since $\deg f = 5$, the code $C = \langle f \rangle$ is a $[11,11-5]_3 = [11,6]_3$ code.

Let's compute its minimum distance.

Using `sage` we can get the coefficients of a polynomial as follows:

In [67]:
gg.coefficients(sparse=False)

[2, 0, 1, 2, 1, 1]

We'll use these coefficients to build the code as a vector space.

In [74]:
V = VectorSpace(k,6)
S = V.subspace([ (T^i * g).coefficients(sparse=False) for i in range(5) ])

TypeError: each element of basis must be in the ambient vector space